# Activity: Comparing Imbalanced Classifiers

In this activity, you’ll fit various balanced and imbalanced models to small business loan data. You’ll then compare the results by using the metrics that you’ve learned.

## Overview

The U.S. Small Business Administration (SBA) is a government agency that exists to support the creation and growth of small companies. The SBA accomplishes this growth in several ways, one of which involves lending to these firms.

The dataset for this activity contains information about actual small business loans that the SBA has issued. This dataset contains the following columns:

- “Year”: The fiscal year of the loan application.

- “Month”: The month of the fiscal year.

- “Amount”: The issued loan amount.

- “Term”: The term of the loan, in months.

- “Zip”: The borrower’s zip code.

- “CreateJob”: The number of jobs that were created by using the loan.

- “NoEmp”: The number of business employees.

- “RealEstate”: Whether the loan is backed by real estate.

- “RevLineCr”: Whether the loan is a revolving line of credit.

- “UrbanRural”: The location type of the borrower.

- “Default”: Whether the borrower defaulted on the loan (1) or not (0).

This dataset is imbalanced. Failing to repay a loan (that is, when the “Default” value equals 1) occurred rarely compared to the number of loans that borrowers successfully repaid.

Using some of these variables as features, you need to try various models to find the one which can best predict which SBA loans are most likely to default.

## Files

Use the Jupyter notebook file in the `Unsolved` folder to write your code. The `Resources` folder contains the CSV file that you’ll import.

## Instructions

1. Read in the CSV file from the `Resources` folder into a Pandas DataFrame.

2. Create a Series named `y` that contains the data from the "Default" column of the original DataFrame. Note that this Series will contain the labels. Create a new DataFrame named `X` that contains the remaining columns from the original DataFrame. Note that this DataFrame will contain the features.

3. Split the features and labels into training and testing sets, and `StandardScaler` your `X` data.

4. Check the magnitude of imbalance in the dataset by viewing the number of distinct values (`value_counts`) for the labels. 

5. Fit two versions of a random forest model to the data: the first, a regular `RandomForest` classifier, and the second, a `BalancedRandomForest` classifier.

6. Resample and fit the training data by one additional method for imbalanced data, such as `RandomOverSampler`, undersampling, or a synthetic technique.

7. Print the confusion matrixes, accuracy scores, and classification reports for the three different models.

8. Evaluate the effectiveness of `RandomForest`, `BalancedRandomForest`, and your one additional imbalanced classifier for predicting the minority class. 
    * Answer the following question: Does the model generated using one of the resampled classifiers more accurately flag all the loans that eventually defaulted?


In [1]:
# Import the required modules
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.metrics import classification_report_imbalanced

## Step 1: Read in the CSV file from the `Resources` folder into a Pandas DataFrame. 

In [2]:
# Read the sba_loans.csv file from the Resources folder into a Pandas DataFrame
loans_df = pd.read_csv(
    Path('../Resources/sba_loans.csv')
)

# Review the DataFrame
loans_df.head()

,Year,Month,Amount,Term,Zip,CreateJob,NoEmp,RealEstate,RevLineCr,UrbanRural,Default
0,2001,11,32812,36,92801,0,1,0,1,0,0
1,2001,4,30000,56,90505,0,1,0,1,0,0
2,2001,4,30000,36,92103,0,10,0,1,0,0
3,2003,10,50000,36,92108,0,6,0,1,0,0
4,2006,7,343000,240,91345,3,65,1,0,2,0


## Step 2: Create a Series named `y` that contains the data from the "Default" column of the original DataFrame. Note that this Series will contain the labels. Create a new DataFrame named `X` that contains the remaining columns from the original DataFrame. Note that this DataFrame will contain the features.

In [3]:
# Split the data into X (features) and y (lables)

# The y variable should focus on the Default column
y = loans_df['Default']

# The X variable should include all features except the Default column
X = loans_df.drop(columns=['Default'])


### Step 3: Split the features and labels into training and testing sets, and `StandardScaler` your X data.

In [4]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [5]:
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Step 4: Check the magnitude of imbalance in the data set by viewing  the number of distinct values  (`value_counts`) for the labels. Scale the data.

In [6]:
# Count the distinct values in the original labels data
y_train.value_counts()

Default
0    1063
1      96
Name: count, dtype: int64

## Step 5: Fit two versions of a random forest model to the data: the first, a regular `RandomForest` classifier, and the second, a `BalancedRandomForest` classifier.

In [7]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=1)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
rf_predictions = rf_model.predict(X_test_scaled)

In [8]:
# Import BalancedRandomForestClassifier from imblearn
from imblearn.ensemble import BalancedRandomForestClassifier

# Instantiate a BalancedRandomForestClassifier instance
brf = BalancedRandomForestClassifier()

# Fit the model to the training data
brf.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,False
,oob_score,False


In [9]:
# Predict labels for testing features
brf_predictions = brf.predict(X_test_scaled)

## Step 6: Resample and fit the training data by one additional method for imbalanced data, such as `RandomOverSampler`, undersampling, or a synthetic technique. Re-esimate by `RandomForest`.

In [10]:
# Import SMOTE from imblearn
from imblearn.over_sampling import SMOTE

# Instantiate the SMOTE model instance
smote_sampler = SMOTE(random_state=1, sampling_strategy='auto')

# Fit the SMOTE model to the training data
X_resampled, y_resampled = smote_sampler.fit_resample(X_train, y_train)

# Fit the RandomForestClassifier on the resampled data
model_resampled_rf = RandomForestClassifier()
model_resampled_rf.fit(X_resampled, y_resampled)

# Generate predictions based on the resampled data model
rf_resampled_predictions = model_resampled_rf.predict(X_test)

## Step 7: Print the confusion matrixes, accuracy scores, and classification reports for the three different models.

In [11]:
# Print the confusion matrix for RandomForest on the original data
confusion_matrix(y_test, rf_predictions)

array([[337,  11],
       [ 13,  26]])

In [12]:
# Print the confusion matrix for balanced random forest data
confusion_matrix(y_test, brf_predictions)

array([[320,  28],
       [  7,  32]])

In [13]:
# Print the confusion matrix for RandomForest on the resampled data
confusion_matrix(y_test, rf_resampled_predictions)

array([[330,  18],
       [  8,  31]])

In [14]:
# Print the accuracy score for the original data
baso = balanced_accuracy_score(y_test, rf_predictions)
print(baso)

0.8175287356321839


In [15]:
# Print the accuracy score for the resampled data
basr = balanced_accuracy_score(y_test, brf_predictions)
print(basr)

0.870026525198939


In [16]:
# Print the accuracy score for the resampled data
basrs = balanced_accuracy_score(y_test, rf_resampled_predictions)
print(basrs)

0.8715738284703802


In [17]:
# Print the classification report for the original data
print(classification_report_imbalanced(y_test, rf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.96      0.97      0.67      0.97      0.80      0.67       348
          1       0.70      0.67      0.97      0.68      0.80      0.63        39

avg / total       0.94      0.94      0.70      0.94      0.80      0.66       387



In [18]:
# Print the classification report for the resampled data
print(classification_report_imbalanced(y_test, brf_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.92      0.82      0.95      0.87      0.76       348
          1       0.53      0.82      0.92      0.65      0.87      0.75        39

avg / total       0.93      0.91      0.83      0.92      0.87      0.76       387



In [19]:
# Print the classification report for the resampled data
print(classification_report_imbalanced(y_test, rf_resampled_predictions))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.95      0.79      0.96      0.87      0.77       348
          1       0.63      0.79      0.95      0.70      0.87      0.74        39

avg / total       0.94      0.93      0.81      0.94      0.87      0.76       387



## Step 8: Evaluate the effectiveness of `RandomForest`, `BalancedRandomForest`, and your one additional imbalanced classifier for predicting the minority class. 

### Answer the following question: Does the model generated using one of the imbalanced methods more accurately flag all the loans that eventually defaulted?

**Question:** Does the model generated using one of the imbalanced methods more accurately flag all the loans that eventually defaulted?
    
**Answer:** Overall, both resampled models in this example perform better at identifying more of the eventual loan defaults. We can see this by looking at the increase recall for the “default” or “`1`” category in the two imbalanced models, when compared to the  original random forest model.

A higher recall for this category means that of all the loans that actually were in default, how many did this model correctly catch? A higher recall for a model means it’s going to do a better job at making sure any potential defaults are not missed.

However, the higher recall for these two imbalanced models comes at a cost: a greater tendency to flag a loan as a potential default, even when it does not. This is evidenced by a lower precision for these two models. If precision looks at, of all those loans the model predicted as default, how many of them actually were defaults, then a lower precision value means that the model is making a lot of false positives; predicting a default when there isn’t actually one. 

This illustrates the main tradeoff when using imbalanced versions of machine learning models. If you really care about identifying those faulty loans (or whatever you’re trying to predict), and the cost of failing to identify a faulty loan is very high, then maybe an imbalanced model with lower precision is worth it. After all, we can always find another business to lend to, but if that business defaults, it’s very costly to us as a lender. 

If on the other hand, you have a situation in which the costs of misclassification are the same either way—if failing to correctly identify a `1` has the same practical cost as failing to correctly identify a `0`—then we may be better off with the overall higher accuracy of a standard machine learning model.
